<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-23 09:55:21
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.41 C
-------------------
Today PnL: 45.80 K (0.32%)
Current PnL: -32.96 L (-20.6%)
CY Booked + Current PnL: -17.28 L (-10.8%)
-------------------
Total profit:  1.31 L
Total loss:  -34.27 L
-------------------
Total Booked + Current PnL: 9.70 L (7.29%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.08%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 99.58 L (70.38%)
Deployed:  1.33 C
Current:  1.41 C
CAGR/XIRR %: 3.18%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,1.47,3.28,17.71,21.57,17921.0,3210.0,101190.0,397.04,-11.02,59.0,H-SC,6.08,167.0,0.18,0.73,47.67,XR,ATH,FINANCE
18,COALINDIA,-0.01,4.14,12.99,17.67,20355.0,6234.0,156695.0,478.51,24.82,54.0,L-LC,11.39,185.0,0.31,1.13,25.79,XY25,ATH,MINING
77,TTKPRESTIG,0.31,-30.33,43.66,0.09,30656.0,-30561.0,70216.0,770.00,63.07,32.0,M-SC,10.04,250.0,-1.00,0.51,0.31,OX40N,NTT,DURABLES
36,ICICIGI,1.01,3.59,15.20,19.34,31509.0,7185.0,207294.0,2252.93,-12.97,62.0,X-MC,1.95,66.0,0.23,1.49,20.10,X40,ATH,INSURANCE
51,MEDANTA,1.72,-4.03,29.28,24.08,35674.0,-5114.0,121836.0,1486.00,-4.72,52.0,X-SC,9.84,87.0,-0.14,0.88,11.59,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TRIDENT,4.19,-28.78,80.45,28.51,52857.0,-26552.0,65702.0,48.00,-8.51,50.0,M-SC,8.45,223.0,-0.50,0.47,12.43,XR,NTT,TEXTILES
7,ATULAUTO,3.70,-14.99,71.23,45.57,119372.0,-29546.0,167586.0,844.00,3629.73,57.0,M-SC,3.90,248.0,-0.25,1.21,26.82,XY24,NTT,AUTO
19,COFFEEDAY,2.85,-44.50,143.75,35.27,90583.0,-50535.0,63014.0,80.00,-57.75,51.0,L-SC,14.34,266.0,-0.56,0.45,54.23,XR,NTT,HOTELS
26,FINCABLES,2.31,2.62,102.61,107.91,155461.0,3863.0,151507.0,1641.55,-7.66,59.0,M-SC,8.14,220.0,0.02,1.09,13.98,OX40N,ATH,CABLES
71,TANLA,1.97,-47.67,319.85,119.69,453157.0,-129083.0,141678.0,1943.92,-47.02,42.0,H-SC,18.72,147.0,-0.28,1.02,11.92,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.70,-9.45,48.25,34.24,136708.0,-29555.0,283333.0,1972.00,-28.88,21.0,X-LC,5.41,3.0,-0.22,2.04,0.00,X40,NTT,IT
23,DIXON,-1.69,-24.53,73.92,31.26,128739.0,-56608.0,174160.0,18931.72,-66.27,41.0,X-MC,24.69,54.0,-0.44,1.25,6.88,X40N,ATH,IT
43,IRCTC,-1.48,-23.38,99.61,52.93,206446.0,-63254.0,207254.0,1269.00,-43.32,54.0,H-MC,12.00,119.0,-0.31,1.49,5.24,XY24,BTT,TRAVEL
25,EASEMYTRIP,-1.40,-49.39,187.90,45.70,168135.0,-87334.0,89481.0,26.40,-7.12,72.0,M-SC,19.06,204.0,-0.52,0.64,47.43,XY24,NTT,TRAVEL
84,WIPRO,-1.32,-15.06,102.80,72.27,147964.0,-25514.0,143934.0,420.00,-23.15,24.0,M-LC,6.90,98.0,-0.17,1.04,0.00,XR,NTT,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,2.31,2.62,102.61,107.91,155461.0,3863.0,151507.0,1641.55,-7.66,59.0,M-SC,8.14,220.0,0.02,1.09,13.98,OX40N,ATH,CABLES
77,TTKPRESTIG,0.31,-30.33,43.66,0.09,30656.0,-30561.0,70216.0,770.00,63.07,32.0,M-SC,10.04,250.0,-1.00,0.51,0.31,OX40N,NTT,DURABLES
72,TATAELXSI,-0.79,-30.53,90.04,32.03,82468.0,-40245.0,91590.0,9161.00,-13.14,34.0,H-SC,8.54,158.0,-0.49,0.66,2.59,OX40N,NTT,IT
66,SIS,0.20,-31.70,77.54,21.26,58804.0,-35195.0,75837.0,528.00,1779.27,30.0,H-SC,4.61,168.0,-0.60,0.55,4.55,OX40N,NTT,MISC
40,INDIGOPNTS,0.17,-32.83,48.90,0.02,57337.0,-57305.0,117254.0,1408.00,78.57,27.0,M-SC,10.38,222.0,-1.00,0.84,1.99,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,0.72,21.88,19.13,45.2,44707.0,41958.0,233700.0,1856.0,39.57,66.0,X-MC,6.76,79.0,0.94,1.68,30.17,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.01,4.14,12.99,17.67,20355.0,6234.0,156695.0,478.51,24.82,54.0,L-LC,11.39,185.0,0.31,1.13,25.79,XY25,ATH,MINING
26,FINCABLES,2.31,2.62,102.61,107.91,155461.0,3863.0,151507.0,1641.55,-7.66,59.0,M-SC,8.14,220.0,0.02,1.09,13.98,OX40N,ATH,CABLES
50,MASFIN,1.47,3.28,17.71,21.57,17921.0,3210.0,101190.0,397.04,-11.02,59.0,H-SC,6.08,167.0,0.18,0.73,47.67,XR,ATH,FINANCE
1,ABB,-0.17,9.66,32.74,45.57,93930.0,25277.0,286896.0,7934.00,-29.53,67.0,H-MC,6.78,125.0,0.27,2.06,27.40,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.17,9.66,32.74,45.57,93930.0,25277.0,286896.0,7934.00,-29.53,67.0,H-MC,6.78,125.0,0.27,2.06,27.40,AR,NTT,ELECTRICAL
50,MASFIN,1.47,3.28,17.71,21.57,17921.0,3210.0,101190.0,397.04,-11.02,59.0,H-SC,6.08,167.0,0.18,0.73,47.67,XR,ATH,FINANCE
26,FINCABLES,2.31,2.62,102.61,107.91,155461.0,3863.0,151507.0,1641.55,-7.66,59.0,M-SC,8.14,220.0,0.02,1.09,13.98,OX40N,ATH,CABLES
39,INDIAMART,0.30,-5.80,119.44,106.72,138774.0,-7149.0,116187.0,4810.62,-57.42,47.0,H-SC,7.97,140.0,-0.05,0.84,16.23,AR,ATH,MISC
85,ZYDUSLIFE,0.77,-4.29,41.70,35.63,83646.0,-8983.0,200591.0,1286.17,-16.67,52.0,H-MC,5.16,120.0,-0.11,1.44,11.67,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.70,-9.45,48.25,34.24,136708.0,-29555.0,283333.0,1972.00,-28.88,21.0,X-LC,5.41,3.0,-0.22,2.04,0.00,X40,NTT,IT
33,HCLTECH,-0.45,-7.20,33.44,23.83,75076.0,-17414.0,224510.0,1908.19,-2.40,24.0,X-LC,3.86,8.0,-0.23,1.61,7.71,X40,ATH,IT
73,TCS,-0.19,-23.15,60.81,23.58,202166.0,-100173.0,332456.0,4311.59,-33.46,27.0,X-LC,6.17,2.0,-0.50,2.39,0.12,X40,ATH,IT
8,AWL,0.59,-35.28,142.88,57.19,301580.0,-115055.0,211072.0,485.00,-63.00,29.0,X-SC,17.83,81.0,-0.38,1.52,0.59,XY24,NTT,FMCG
53,PGHH,-0.05,-13.86,53.26,32.01,97956.0,-29600.0,183920.0,17616.87,-41.76,32.0,X-MC,9.55,57.0,-0.30,1.32,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,0.72,2.05,18.34,20.77,43822.0,4798.0,238942.0,1644.00,-9.55,52.0,X-MC,0.65,64.0,0.11,1.72,9.60,X40N,NTT,BREWERIES
21,DABUR,-0.28,-0.63,44.61,43.70,109964.0,-1567.0,246501.0,735.00,-9.86,48.0,X-MC,1.28,70.0,-0.01,1.77,14.32,XY24,BTT,FMCG
36,ICICIGI,1.01,3.59,15.20,19.34,31509.0,7185.0,207294.0,2252.93,-12.97,62.0,X-MC,1.95,66.0,0.23,1.49,20.10,X40,ATH,INSURANCE
65,SIEMENS,1.86,-12.25,43.04,25.51,70282.0,-22800.0,163295.0,4671.50,37.11,60.0,H-LC,2.01,51.0,-0.32,1.17,20.34,AR,ATH,ELECTRICAL
75,TMPV,0.90,-15.24,57.32,33.33,158280.0,-49666.0,276134.0,600.00,-81.44,61.0,X-LC,2.68,4.0,-0.31,1.99,12.89,XY24,NTT,AUTO


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PGHH,-0.05,-13.86,53.26,32.01,97956.0,-29600.0,183920.0,17616.87,-41.76,32.0,X-MC,9.55,57.0,-0.30,1.32,0.00,X40,ATH,FMCG
42,INFY,-1.70,-9.45,48.25,34.24,136708.0,-29555.0,283333.0,1972.00,-28.88,21.0,X-LC,5.41,3.0,-0.22,2.04,0.00,X40,NTT,IT
73,TCS,-0.19,-23.15,60.81,23.58,202166.0,-100173.0,332456.0,4311.59,-33.46,27.0,X-LC,6.17,2.0,-0.50,2.39,0.12,X40,ATH,IT
9,BAJAJHFL,0.24,-24.38,105.60,55.47,194279.0,-59310.0,183976.0,181.50,-35.17,36.0,X-MC,17.60,60.0,-0.31,1.32,0.30,X40N,ATH,FINANCE
58,RELAXO,-0.49,-53.87,230.15,52.31,154122.0,-78194.0,66966.0,1176.00,-50.92,39.0,X-SC,14.07,92.0,-0.51,0.48,0.39,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-0.52,-32.20,110.68,42.84,48781.0,-20932.0,44074.0,424.00,-55.37,44.0,X-SC,17.88,80.0,-0.43,0.32,6.02,XY24,NTT,MISC
58,RELAXO,-0.49,-53.87,230.15,52.31,154122.0,-78194.0,66966.0,1176.00,-50.92,39.0,X-SC,14.07,92.0,-0.51,0.48,0.39,X40N,NTT,FOOTWEAR
12,BATAINDIA,0.54,-47.35,156.78,35.20,106217.0,-60921.0,67749.0,2096.00,-11.19,34.0,X-SC,21.79,91.0,-0.57,0.49,0.54,X40,NTT,FOOTWEAR
51,MEDANTA,1.72,-4.03,29.28,24.08,35674.0,-5114.0,121836.0,1486.00,-4.72,52.0,X-SC,9.84,87.0,-0.14,0.88,11.59,XY24,NTT,HEALTHCARE
35,HONAUT,1.08,-23.30,86.86,43.32,108506.0,-37952.0,124920.0,58357.33,-32.62,40.0,X-SC,11.57,90.0,-0.35,0.90,1.43,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-0.05,-13.91,38.29,19.06,112135.0,-47308.0,292858.0,452.00,-52.69,50.0,X-LC,17.89,1.0,-0.42,2.11,6.04,X40,NTT,FMCG
73,TCS,-0.19,-23.15,60.81,23.58,202166.0,-100173.0,332456.0,4311.59,-33.46,27.0,X-LC,6.17,2.0,-0.50,2.39,0.12,X40,ATH,IT
42,INFY,-1.70,-9.45,48.25,34.24,136708.0,-29555.0,283333.0,1972.00,-28.88,21.0,X-LC,5.41,3.0,-0.22,2.04,0.00,X40,NTT,IT
75,TMPV,0.90,-15.24,57.32,33.33,158280.0,-49666.0,276134.0,600.00,-81.44,61.0,X-LC,2.68,4.0,-0.31,1.99,12.89,XY24,NTT,AUTO
33,HCLTECH,-0.45,-7.20,33.44,23.83,75076.0,-17414.0,224510.0,1908.19,-2.40,24.0,X-LC,3.86,8.0,-0.23,1.61,7.71,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.32,-31.85,95.00,32.89,49986.0,-24590.0,52617.0,1800.00,-347.90,53.0,L-MC,11.31,258.0,-0.49,0.38,44.90,XR,NTT,BANKS
67,SONACOMS,1.05,-6.98,49.53,39.09,46607.0,-7061.0,94098.0,804.02,-26.28,65.0,M-MC,9.14,193.0,-0.15,0.68,33.12,AR,ATH,AUTO
50,MASFIN,1.47,3.28,17.71,21.57,17921.0,3210.0,101190.0,397.04,-11.02,59.0,H-SC,6.08,167.0,0.18,0.73,47.67,XR,ATH,FINANCE
18,COALINDIA,-0.01,4.14,12.99,17.67,20355.0,6234.0,156695.0,478.51,24.82,54.0,L-LC,11.39,185.0,0.31,1.13,25.79,XY25,ATH,MINING
7,ATULAUTO,3.70,-14.99,71.23,45.57,119372.0,-29546.0,167586.0,844.00,3629.73,57.0,M-SC,3.90,248.0,-0.25,1.21,26.82,XY24,NTT,AUTO


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,-1.40,-49.39,187.90,45.70,168135.0,-87334.0,89481.0,26.40,-7.12,72.0,M-SC,19.06,204.0,-0.52,0.64,47.43,XY24,NTT,TRAVEL
1,ABB,-0.17,9.66,32.74,45.57,93930.0,25277.0,286896.0,7934.00,-29.53,67.0,H-MC,6.78,125.0,0.27,2.06,27.40,AR,NTT,ELECTRICAL
7,ATULAUTO,3.70,-14.99,71.23,45.57,119372.0,-29546.0,167586.0,844.00,3629.73,57.0,M-SC,3.90,248.0,-0.25,1.21,26.82,XY24,NTT,AUTO
82,VOLTAS,0.72,21.88,19.13,45.20,44707.0,41958.0,233700.0,1856.00,39.57,66.0,X-MC,6.76,79.0,0.94,1.68,30.17,XY25,NTT,AC
67,SONACOMS,1.05,-6.98,49.53,39.09,46607.0,-7061.0,94098.0,804.02,-26.28,65.0,M-MC,9.14,193.0,-0.15,0.68,33.12,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.68
1,25,45.39
2,50,77.31


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.15
MC    30.09
LC    24.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.96
X40      23.98
X40N     14.78
AR        9.01
XY25      8.80
XR        8.53
OX40N     7.10
SR        0.84
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.64
H-SC    22.13
X-LC    21.42
X-SC    11.56
M-SC    10.71
H-MC     4.99
M-MC     2.08
H-LC     1.17
L-LC     1.13
M-LC     1.04
L-SC     0.75
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.56,-14.02,52.59
FINANCE,13.04,-15.17,60.49
IT,11.52,-36.87,105.46
MISC,6.92,-35.27,90.61
ELECTRICAL,6.27,-7.61,46.86
PAINTS,5.01,-32.59,52.29
INSURANCE,4.83,-0.94,35.48
PHARMA,4.10,-3.04,35.47
AUTO,3.88,-16.04,60.29


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3380310.0,21
X40,1387693.0,15
AR,1318556.0,10
XR,1305351.0,12
X40N,1064553.0,10
OX40N,804696.0,10
XY25,394699.0,6
SR,301820.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3715943.0,24
M-SC,1427177.0,14
X-MC,1364239.0,15
X-LC,1296007.0,12
X-SC,1198739.0,10
H-MC,384022.0,3
L-SC,186250.0,2
M-LC,147964.0,1
M-MC,96714.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1269447.0      6
           AR         949514.0      5
           XR         840527.0      7
M-SC       XY24       821915.0      6
X-LC       X40        670868.0      6
X-SC       XY24       614449.0      4
X-MC       X40        535876.0      7
           X40N       431477.0      4
X-SC       X40N       403341.0      4
H-SC       OX40N      354635.0      4
M-SC       OX40N      354394.0      5
H-SC       SR         301820.0      2
X-LC       XY24       268881.0      2
           X40N       229735.0      2
H-MC       XY24       206446.0      1
X-MC       XY24       199172.0      2
           XY25       197714.0      2
X-SC       X40        180949.0      2
H-MC       AR         177576.0      2
M-SC       XR         176291.0      2
M-LC       XR         147964.0      1
X-LC       XY25       126523.0      2
L-SC       OX40N       95667.0      1
           XR          90583.0      1
M-SC       AR          74577.0      1
H-LC       AR          70282.0      1
M-MC       XY25        50107.0      1
L-MC       XR          49986.0      1
M-MC       AR          46607.0      1
L-LC       XY25        20355.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
